<a href="https://colab.research.google.com/github/adimnicky/Capstone-Project/blob/main/Capstone_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain_community
!pip install replicate
!pip install pandas

In [ ]:
from langchain_community.llms import Replicate
import os
from google.colab import userdata
import pandas as pd

# Set the API token
api_token = userdata.get('api_token')
os.environ["REPLICATE_API_TOKEN"] = api_token
# Model setup
model = "ibm-granite/granite-3.3-8b-instruct"
output = Replicate(
model=model,
replicate_api_token=api_token,
)

# Load dataset from CSV
df = pd.read_csv("Tweets.csv")

In [ ]:
# Define a function to split the list into batches
def batch_list(data, batch_size):
    for i in range(0, len(data), batch_size):
        yield data[i:i + batch_size]

# Define the batch size (adjust as needed based on the model's context window)
batch_size = 100 # You can adjust this number

# Create batches of reviews
# Assuming 'text' column contains the reviews
review_batches = list(batch_list(df['text'], batch_size))

# Process each batch and store the responses
responses = []
for i, batch in enumerate(review_batches):
    # Use the prompt_template defined earlier
    reviews_text_batch = "\n".join([f"Review {j+1}: {review}" for j, review in enumerate(batch)])
    # Check if prompt_template is defined before using it
    if 'prompt_template' in locals() or 'prompt_template' in globals():
        prompt_batch = prompt_template.format(reviews_text_batch=reviews_text_batch)
    else:
        # Define a default prompt_template if not found
        prompt_batch = f"Analyze the following reviews:\n{reviews_text_batch}"


    print(f"Processing batch {i+1}/{len(review_batches)}...")
    response_batch = output.invoke(prompt_batch)
    responses.append(response_batch)

# Combine the responses (this will depend on the desired output format)
# For now, let's just print each response
for i, response in enumerate(responses):
    print(f"\nResponse for batch {i+1}:\n")
    print(response)

Processing batch 1/147...
Processing batch 2/147...
Processing batch 3/147...
Processing batch 4/147...
Processing batch 5/147...
Processing batch 6/147...
Processing batch 7/147...
Processing batch 8/147...
Processing batch 9/147...
Processing batch 10/147...
Processing batch 11/147...
Processing batch 12/147...
Processing batch 13/147...
Processing batch 14/147...
Processing batch 15/147...
Processing batch 16/147...
Processing batch 17/147...
Processing batch 18/147...
Processing batch 19/147...
Processing batch 20/147...
Processing batch 21/147...
Processing batch 22/147...
Processing batch 23/147...
Processing batch 24/147...
Processing batch 25/147...
Processing batch 26/147...
Processing batch 27/147...
Processing batch 28/147...
Processing batch 29/147...
Processing batch 30/147...
Processing batch 31/147...
Processing batch 32/147...
Processing batch 33/147...
Processing batch 34/147...
Processing batch 35/147...
Processing batch 36/147...
Processing batch 37/147...
Processing

In [ ]:
# Define refined prompt
refined_prompt = f"""
Classify these reviews as positive, negative, or mixed, and tag
relevant categories (late flight, or rude service):
{response_batch}
"""
# Invoke the model with the example prompt
response = output.invoke(refined_prompt)
# Print the response
print("Granite Model Refined Response:\n")
print(response)

Granite Model Refined Response:

**Classification and Tagging:**

**Negative Reviews:**

1. **Late Flight:** Reviews 1, 4, 5, 7, 13, 17, 27 - These express dissatisfaction due to flight cancellations or significant delays, often linked to weather or mechanical issues. Tags: #LateFlight, #PoorCommunication

2. **Rude Service:** Reviews 10, 21, 22, 31, 39 - Highlight issues with customer service, including rudeness, long wait times, and unhelpful representatives. Tags: #RudeService, #PoorCustomerService

3. **Lost Luggage:** Reviews 2, 9, 16 - Describe long waits for luggage following lengthy delays, causing additional passenger distress. Tags: #LostLuggage, #Inconvenience

4. **Inefficient Rebooking:** Reviews 6, 24, 30 - Complain about the difficulty in rebooking flights due to busy lines or inaccessible staff. Tags: #InefficientRebooking, #CommunicationIssues

5. **Inconsistent Service:** Reviews 24, 28 - Mention faulty amenities and a lack of accommodations for stranded passengers, i

In [ ]:
# Define the prompt to complete the task in 2 steps
multitask_prompt = f"""
Complete the task in 2 steps.
Step 1: Classify these reviews as positive, negative, or mixed.
Step 2: For each review, identify relevant categories: battery
life, screen quality, or performance.
{response_batch}
"""
response = output.invoke(multitask_prompt)
print("Granite Model Response:\n")
print(response)

Granite Model Response:

**Step 1: Classification**

- Negative Reviews: 1, 4, 5, 6, 7, 9, 10, 13, 16, 17, 21, 22, 24, 27, 28, 30, 31, 39
- Positive Reviews: 20, 26, 29, 33, 36
- Mixed Reviews: 2, 11, 18 (These contain both criticism and appreciation)

**Step 2: Identify Relevant Categories**

**Negative Experiences:**

1. **Delays and Cancellations** (1, 4, 5, 7, 13, 17, 27): Complaints about flight disruptions and lack of timely updates.
2. **Poor Customer Service** (10, 21, 22, 31, 39): Rude staff, long wait times, and unhelpful representatives.
3. **Lost Luggage and Inconvenience** (2, 9, 16): Long waits for luggage and additional stress caused by delays.
4. **Inefficient Rebooking Process** (6, 24, 30): Difficulty in rebooking flights due to communication barriers.
5. **Inconsistent Service and Amenities** (24, 28): Faulty in-flight amenities and insufficient support for stranded passengers.
6. **Arbitrary Changes** (7, 19): Sudden itinerary changes without proper notice or compen

In [ ]:
# Define the example to guide the model
formatted_prompt = f"""
Classify these reviews as Positive, Negative, or Mixed, and tag
relevant categories. Use this format:
- Sentiment: [Sentiment]
- Categories: [Categories].
{response_batch}
"""
# Invoke the model with prompt
response = output.invoke(formatted_prompt)
# Print the response
print("Granite Model Formatted Response:\n")
print(response)

Granite Model Formatted Response:

- Sentiment: Mixed
  - Categories: Delays and Cancellations, Poor Customer Service, Lost Luggage and Inconvenience, Inefficient Rebooking Process, Inconsistent Service and Amenities, Arbitrary Changes, Positive Flight Experience, Effective Problem Resolution, Efficient Rebooking, Assistance and Communication.

The reviews present a mixed sentiment regarding American Airlines, with passengers reporting both dissatisfaction and positive experiences. Negative aspects include frequent flight delays and cancellations, poor customer service, lost luggage, inconvenient rebooking processes, inconsistent service quality, and sudden itinerary changes. On the positive side, some reviews highlight good interactions with cabin crew, efficient problem resolution, successful rebooking experiences, and helpful communication from the airline.

Note: Some reviews, like those numbered 11 and others without explicit sentiment, remain unclassified due to insufficient info